# 3. Create Matrix Tissues / TFs

In [56]:
source("functions_promoter_data_preparation.r")
tissues <- get_tissue_names("/project/pauline_association_plots_data/experiments_single_data_2/bed/raw/")
source("enhancer_analysis/functions_tf_annotation.r")

Installing package into ‘/home/huebnerp/R/x86_64-pc-linux-gnu-library/4.2’
(as ‘lib’ is unspecified)

Warning message in install.packages("dplyr"):
“installation of package ‘dplyr’ had non-zero exit status”
'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org


Bioconductor version 3.15 (BiocManager 1.30.18), R 4.2.1 (2022-06-23)

Warning message:
“package(s) not installed when version(s) same as current; use `force = TRUE` to
  re-install: 'FANTOM3and4CAGE'”
Installation paths not writeable, unable to update packages
  path: /pkg/R-4.2.1-0/lib/R/library
  packages:
    AnnotationForge, BBmisc, BH, BiocManager, BiocParallel, Biostrings, Boruta,
    C50, Cairo, ChIPseeker, Ckmeans.1d.dp, ComplexHeatmap, Cubist, DEoptimR,
    DOSE, DT, DelayedMatrixStats, ExomeDepth, FNN, Formula, GGally,
    GenomeInfoDb, GenomicFeatures, HDF5Array, Hmisc, IRanges, KernSmooth,
    Lahman, MASS, M

see document "3_tissues_CAGEr_analysisHUMAN.ipynb", bedfile named 'All.samples.tagClusters.qLow0.1_qUp0.9.bed' should have been created with the following command:

```command
make bed file for all tissues
exportToBed( object = tissuesCAGEset, what = "tagClusters"
           , qLow = 0.1, qUp = 0.9, oneFile = TRUE)
```

## 1. Data Preparation
### 1.1 Subset Bedfile in Separate Ones for Tissues
the file 'All.samples.tagClusters.qLow0.1_qUp0.9.bed' has all the tissues, now we make separate files


In [ ]:
# make separate bedfile for each tissue

awk_command <- 'awk \'/track/{x=(substr($2,7)) ".bed";}{print > x ;}\' All.samples.tagClusters.qLow0.1_qUp0.9.bed'
system(awk_command, intern = TRUE) %>% cat()

In [33]:
setwd('/project/pauline_association_plots/code/01_promoter-celltype_matrix/experiments_single/bed/raw')


### 1.2 Change Start and Stop to Promoter Regions in Every File

In [55]:
# change regions (ctss clusters) to promoter regions and save files

bp_upstream = 200
bp_downstream = 50

define_promoter_size(bp_upstream, bp_downstream, tissues)

### 1.3 Generate FASTA Files from these Bedfiles

In [20]:
# unchanged promoters

for (i in 1:length(tissues)) {
    system(
        paste("bedtools getfasta -fi hg18.fa -bed experiments_single_2/bed/raw/", 
              tissues[i], ".bed -s -fo experiments_single/fasta/raw/", 
              tissues[i] ,".fa", "\n", sep=""), intern = TRUE)
    }

bedtools getfasta -fi hg18.fa -bed experiments_single/bed/raw/adipose_1.bed -s -fo experiments_single/fasta/adipose_1.fa
bedtools getfasta -fi hg18.fa -bed experiments_single/bed/raw/adipose_2.bed -s -fo experiments_single/fasta/adipose_2.fa
bedtools getfasta -fi hg18.fa -bed experiments_single/bed/raw/adipose_3.bed -s -fo experiments_single/fasta/adipose_3.fa
bedtools getfasta -fi hg18.fa -bed experiments_single/bed/raw/adipose_4.bed -s -fo experiments_single/fasta/adipose_4.fa
bedtools getfasta -fi hg18.fa -bed experiments_single/bed/raw/adrenal_gland_1.bed -s -fo experiments_single/fasta/adrenal_gland_1.fa
bedtools getfasta -fi hg18.fa -bed experiments_single/bed/raw/adrenal_gland_2.bed -s -fo experiments_single/fasta/adrenal_gland_2.fa
bedtools getfasta -fi hg18.fa -bed experiments_single/bed/raw/adrenal_gland_3.bed -s -fo experiments_single/fasta/adrenal_gland_3.fa
bedtools getfasta -fi hg18.fa -bed experiments_single/bed/raw/blood_1.bed -s -fo experiments_single/fasta/blood_1.fa


In [56]:
# changed promoter areas

for (i in 1:length(tissues)) {
    system(
        paste("bedtools getfasta -fi hg18.fa -bed experiments_single_2/bed/promoter_m200_p50/", 
              tissues[i], ".bed -s -fo experiments_single/fasta/promoter_m200_p50/", 
              tissues[i] ,".fa", "\n", sep=""), intern = TRUE)
    }

bedtools getfasta -fi hg18.fa -bed experiments_single/bed/promoter_m200_p50/adipose_1.bed -s -fo experiments_single/fasta/promoter_m200_p50/adipose_1.fa
bedtools getfasta -fi hg18.fa -bed experiments_single/bed/promoter_m200_p50/adipose_2.bed -s -fo experiments_single/fasta/promoter_m200_p50/adipose_2.fa
bedtools getfasta -fi hg18.fa -bed experiments_single/bed/promoter_m200_p50/adipose_3.bed -s -fo experiments_single/fasta/promoter_m200_p50/adipose_3.fa
bedtools getfasta -fi hg18.fa -bed experiments_single/bed/promoter_m200_p50/adipose_4.bed -s -fo experiments_single/fasta/promoter_m200_p50/adipose_4.fa
bedtools getfasta -fi hg18.fa -bed experiments_single/bed/promoter_m200_p50/adrenal_gland_1.bed -s -fo experiments_single/fasta/promoter_m200_p50/adrenal_gland_1.fa
bedtools getfasta -fi hg18.fa -bed experiments_single/bed/promoter_m200_p50/adrenal_gland_2.bed -s -fo experiments_single/fasta/promoter_m200_p50/adrenal_gland_2.fa
bedtools getfasta -fi hg18.fa -bed experiments_single/bed/

### 1.4 FIMO Motif Finding and TF Annotation

fimo transfac2021_vertebrate_recommended\(1\).meme spleen.fa --> first run, in next run specify output folder, like here specified
https://meme-suite.org/meme/doc/fimo.html?man_type=web

In [78]:
# fimo files with q-value threshold

for (i in 1:length(tissues)) {
    system(
        paste("fimo --qv-thresh --o experiments_single_2/fimo/qv-thres/", 
              tissues[i], " transfac2021_vertebrate_recommended.meme experiments_single/fasta/promoter_m200_p50/", 
              tissues[i], ".fa", "\n", sep=""), intern = TRUE)
    }

fimo --qv-thresh --o experiments_single/fimo/qv-thres/adipose_1 transfac2021_vertebrate_recommended.meme experiments_single/fasta/promoter_m200_p50/adipose_1.fa
fimo --qv-thresh --o experiments_single/fimo/qv-thres/adipose_2 transfac2021_vertebrate_recommended.meme experiments_single/fasta/promoter_m200_p50/adipose_2.fa
fimo --qv-thresh --o experiments_single/fimo/qv-thres/adipose_3 transfac2021_vertebrate_recommended.meme experiments_single/fasta/promoter_m200_p50/adipose_3.fa
fimo --qv-thresh --o experiments_single/fimo/qv-thres/adipose_4 transfac2021_vertebrate_recommended.meme experiments_single/fasta/promoter_m200_p50/adipose_4.fa
fimo --qv-thresh --o experiments_single/fimo/qv-thres/adrenal_gland_1 transfac2021_vertebrate_recommended.meme experiments_single/fasta/promoter_m200_p50/adrenal_gland_1.fa
fimo --qv-thresh --o experiments_single/fimo/qv-thres/adrenal_gland_2 transfac2021_vertebrate_recommended.meme experiments_single/fasta/promoter_m200_p50/adrenal_gland_2.fa
fimo --qv-

In [79]:
# fimo files with p-value threshold

for (i in 1:length(tissues)) {
    system(
        paste("fimo --o experiments_single_2/fimo/pv-thres/", 
              tissues[i], " transfac2021_vertebrate_recommended.meme experiments_single/fasta/promoter_m200_p50/", 
              tissues[i], ".fa", "\n", sep=""), intern = TRUE
    }

fimo --o experiments_single/fimo/pv-thres/adipose_1 transfac2021_vertebrate_recommended.meme experiments_single/fasta/promoter_m200_p50/adipose_1.fa
fimo --o experiments_single/fimo/pv-thres/adipose_2 transfac2021_vertebrate_recommended.meme experiments_single/fasta/promoter_m200_p50/adipose_2.fa
fimo --o experiments_single/fimo/pv-thres/adipose_3 transfac2021_vertebrate_recommended.meme experiments_single/fasta/promoter_m200_p50/adipose_3.fa
fimo --o experiments_single/fimo/pv-thres/adipose_4 transfac2021_vertebrate_recommended.meme experiments_single/fasta/promoter_m200_p50/adipose_4.fa
fimo --o experiments_single/fimo/pv-thres/adrenal_gland_1 transfac2021_vertebrate_recommended.meme experiments_single/fasta/promoter_m200_p50/adrenal_gland_1.fa
fimo --o experiments_single/fimo/pv-thres/adrenal_gland_2 transfac2021_vertebrate_recommended.meme experiments_single/fasta/promoter_m200_p50/adrenal_gland_2.fa
fimo --o experiments_single/fimo/pv-thres/adrenal_gland_3 transfac2021_vertebrate_

### 1.5 Collapse Overlapping Matches of same TF Binding Motif

In [ ]:
# set file paths fimo files pv threshold

file_paths_fimo_pv <- list()

for (tissue in tissues) {
    file_path <- paste("experiments_single_2/fimo/pv-thres/", tissue, "/fimo.tsv", sep="")
    file_paths_fimo_pv[[tissue]] <- file_path
}

# collapse overlapping sequences

collapse_fimo_function(tissues, file_paths_fimo_pv)

## 2. Make Matrix TISSUES / TFs

In [92]:
# set file paths fimo files pv threshold, collapsed

file_paths_fimo_pv_collapsed <- list()

for (tissue in tissues) {
    file_path <- paste("/project/pauline_association_plots_data/experiments_single_data_2/fimo/pv-thres/0_collapsed/",  
                       tissue, "_collapsed.tsv", sep="")
    file_paths_fimo_pv_collapsed[[tissue]] <- file_path
}

output_file = 'results/matrix_tissues_tfs.tsv'
matrix_pv <- combine_and_write_tfs(tissues, file_paths_fimo_pv_collapsed, output_file)


   motif_id
1:     ACE2
2:     ACE2
3:     ACE2
4:     ACE2
5:     ACE2
6:     ACE2

 ABL1 ACAAT  ACE2 AEBP2  AFP1 AHDC1 
  384   681   971   784    63   171 


Warning message in df_tmp_tf$tissue <- tissues[i]:
“Coercing LHS to a list”


   motif_id
1:     ACE2
2:     ACE2
3:     ACE2
4:     ACE2
5:     ACE2
6:     ACE2

 ABL1 ACAAT  ACE2 AEBP2  AFP1 AHDC1 
  195   352   535   416    27    94 


Warning message in df_tmp_tf$tissue <- tissues[i]:
“Coercing LHS to a list”


   motif_id
1:     ACE2
2:     ACE2
3:     ACE2
4:     ACE2
5:     ACE2
6:     ACE2

 ABL1 ACAAT  ACE2 AEBP2  AFP1 AHDC1 
  170   318   513   360    33    82 


...

Warning message in df_tmp_tf$tissue <- tissues[i]:
“Coercing LHS to a list”


   motif_id
1:     ACE2
2:     ACE2
3:     ACE2
4:     ACE2
5:     ACE2
6:     ACE2

 ABL1 ACAAT  ACE2 AEBP2  AFP1 AHDC1 
  776   633  1061   611   195   404 


Warning message in df_tmp_tf$tissue <- tissues[i]:
“Coercing LHS to a list”


   motif_id
1:     ACE2
2:     ACE2
3:     ACE2
4:     ACE2
5:     ACE2
6:     ACE2

 ABL1 ACAAT  ACE2 AEBP2  AFP1 AHDC1 
  163   153   268   198    22    80 


Warning message in df_tmp_tf$tissue <- tissues[i]:
“Coercing LHS to a list”


In [89]:
head(matrix_pv)

,ABL1,ACAAT,ACE2,AEBP2,AFP1,AHDC1,AHR,AIOLOS,AIRE,ALF,⋯,ZSCAN2,ZSCAN30,ZSCAN4,ZSCAN5,ZSCAN5B,ZSCAN5C,ZTA,ZXDA,ZXDB,ZXDL
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
adipose_1,384,681,971,784,63,171,3801,6123,98,412,⋯,328,12500,722,1214,173,212,266,787,334,53086
adipose_2,195,352,535,416,27,94,1927,3434,54,210,⋯,163,6598,399,638,99,125,131,407,174,57646
adipose_3,170,318,513,360,33,82,1863,3126,52,197,⋯,159,6185,381,564,95,119,138,413,185,53168
adipose_4,192,317,519,376,39,102,1821,3084,49,219,⋯,171,6076,362,564,90,119,141,389,167,54225
adrenal_gland_1,428,819,914,849,56,135,4379,6777,68,457,⋯,370,12918,702,1237,197,229,194,790,360,57347
adrenal_gland_2,274,531,625,537,51,115,2821,4453,60,304,⋯,272,8434,533,778,141,168,150,530,240,73860


In [94]:
ncol(matrix_pv)
write.table(matrix_pv, "results/matrix_promoter_tfs.tsv", 
            sep="\t", row.names=TRUE, col.names=TRUE)

[1] 1588

# Matrix with ctss number

In [91]:
# make tissue list
files = list.files("/project/pauline_association_plots_data/experiments_single_data_2/tsv_all_data/complete/")
excluded_ext <- "_all_data.tsv"
tissues <- sub(paste0(excluded_ext, "$"), "", files)

In [ ]:
library(data.table)

# initialize empty data frame
matrix <- data.frame()


for (tissue in tissues) {

    # load dataframe with cage tags (all data)
    df_all <- read.table(paste('/project/pauline_association_plots_data/experiments_single_data_2/tsv_all_data/complete/', tissue,'_all_data.tsv', sep=''),
                         header=TRUE, sep="\t")
    # load fimo file
    df_fimo <- read.table(paste('/project/pauline_association_plots_data/experiments_single_data_2/fimo/pv-thres/0_collapsed/', tissue, 
                          '_collapsed.tsv', sep=''), header=TRUE, sep='\t')
    
    # Convert df_fimo to a data.table
    setDT(df_fimo)
    
    # make new df with unique combinations for TF and coordinates
    new_fimo <- distinct(df_fimo, motif_id, sequence_name)
    new_fimo <- as.data.frame(new_fimo[, motif_id := str_extract(motif_id, "(?<=_)[^_]+(?=_)")])
    

    # Split the "sequence_name" column into separate columns
    new_fimo <- new_fimo %>%
      separate(sequence_name, into = c("chr", "start_end"), sep = ":", remove = FALSE) %>%
      separate(start_end, into = c("start", "end"), sep = "-") %>%
      mutate(end = sub("\\(.*", "", end))

    head(new_fimo)
    head(df_all)
    
    # Merge fimo and all_data based on matching values in "chr", "start", and "end" columns
    merged_df <- merge(new_fimo, df_all[, c("chr", "start", "end", "nr_ctss")], 
                       by = c("chr", "start", "end"), all.x = TRUE)

    # delete duplicates
    merged_df <- merged_df[!duplicated(merged_df), ]

    # Calculate the sum of "nr_ctss" values per TF
    sum_df <- aggregate(nr_ctss ~ motif_id, data = merged_df, FUN = sum)
    
    # new_colname <- tissue
    colnames(sum_df)[2] <- tissue
    
    # assign tissues as rownames
    rownames(sum_df) <- sum_df[,1]
    sum_df[,1] <- NULL

    matrix <- bind_rows(matrix, data.frame(t(sum_df)))
    }

matrix[is.na(matrix)] <- 0
#write.table(matrix, "/project/pauline_association_plots/code/01_promoter-celltype_matrix/experiments_single/results/matrix_tf_ctss.tsv", 
            #sep="\t", row.names=TRUE, col.names=TRUE)

head(matrix, 30)
nrow(matrix)

Warning message:
“Expected 2 pieces. Additional pieces discarded in 814167 rows [1, 7, 8, 11, 12, 15, 18, 19, 20, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 37, ...].”
Warning message:
“Expected 2 pieces. Additional pieces discarded in 433884 rows [2, 3, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 23, 24, 31, 36, 37, 39, ...].”
Warning message:
“Expected 2 pieces. Additional pieces discarded in 412056 rows [2, 3, 4, 6, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 20, 21, 22, 23, 24, 25, ...].”
Warning message:
“Expected 2 pieces. Additional pieces discarded in 410563 rows [3, 6, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 25, 28, 29, 30, 31, 33, 34, ...].”
Warning message:
“Expected 2 pieces. Additional pieces discarded in 861915 rows [2, 3, 4, 5, 7, 8, 10, 12, 13, 14, 16, 17, 18, 19, 21, 22, 24, 27, 28, 29, ...].”
Warning message:
“Expected 2 pieces. Additional pieces discarded in 571458 rows [1, 2, 3, 6, 7, 8, 9, 10, 11, 12, 13, 14, 18, 21, 23, 26, 27, 32, 33, 35, ...].”
Warning message:
“Expe

In [85]:
head(matrix)

,ABL1,ACAAT,ACE2,AEBP2,AFP1,AHDC1,AHR,AIOLOS,AIRE,ALF,⋯,ZSCAN2,ZSCAN30,ZSCAN4,ZSCAN5,ZSCAN5B,ZSCAN5C,ZTA,ZXDA,ZXDB,ZXDL
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
adipose_1,1290,3564,4687,3907,237,684,14319,16459,381,2093,⋯,1418,26641,2569,5298,940,1000,1315,3931,1739,30054
adipose_2,563,1561,2146,1679,112,315,6413,7735,165,906,⋯,583,12329,1125,2418,427,473,577,1793,833,14917
adipose_3,534,1519,2155,1635,118,292,6251,7481,164,863,⋯,651,12135,1192,2398,438,448,632,1836,842,14503
adipose_4,344,1138,1586,1316,89,251,4580,5624,122,682,⋯,467,8851,886,1675,343,353,386,1283,585,10710
adrenal_gland_1,1586,4579,5122,4713,275,625,17265,19338,314,2457,⋯,1854,30753,2852,6064,1144,1224,1091,4510,1928,35636
adrenal_gland_2,731,2112,2390,2161,154,346,7958,8786,223,1170,⋯,851,14512,1433,2732,534,610,553,2144,883,18251
